<a href="https://colab.research.google.com/github/XescLlabres/tfgFLM/blob/main/HUSE_per_pacient/dadeshuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

IMPUTACIÓ MITJANA I MODA

In [ ]:
df = pd.read_csv('huse.csv')

In [ ]:
df.head()
np.shape(df)

(144, 118)

In [ ]:
# Preprocés: eliminam columnes no desitjades i pacients sense variable predictora
df = df.drop(columns=['CT_PRETACE_DATE', 'TACE_DATE', 'Exam_date1', 'Exam_date2', 'LR 2', 'LR  3', 'LR4', 'LR5', '7-11_CRITERIA_CAT'])
start_index = df[df['ID'] == 8965].index[0]
df = df.loc[:start_index - 1]
df = df.replace(',', '.', regex=True).astype(float)


# Definir el llindar mínim de valors no nuls, en el nostre cas, del 65%
threshold = 0.65
min_non_null_count = int(threshold * len(df))

# Guardar les columnes inicials abans d'eliminar
initial_columns = set(df.columns)

# Filtrar columnas amb menys del 65% de valors no nuls
df = df.dropna(thresh=min_non_null_count, axis=1)

# Obtener les columnas eliminades
removed_columns = initial_columns - set(df.columns)

# Mostrar el nombre i nom de las columnes eliminades
print(f"Nombre de columnes eliminades: {len(removed_columns)}")
print("Nom de les columnes eliminades:", list(removed_columns))


Nombre de columnes eliminades: 21
Nom de les columnes eliminades: ['BCLC', 'Bilirrubina_directa', 'ECOG', 'HIV_+', 'Calcio', 'AntiHBs', 'Encefalopathy', 'Bilirrubina_inderecta', 'otherserology_positivie_result', 'Magnesio', 'TP', 'MELD_SCORE', 'Proteina_total', 'HBsAc', 'TTPa', 'MELDNa_SCORE', 'Fosforo', 'HIV_Serology', 'DOSE _DLP', 'Cloruro', 'LDH']


In [ ]:
df.tail()

,ID,CLAVE,RECIST,GENDER,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
126,8942.0,8942.0,1.0,1.0,83.0,194.0,75.0,1.60,29.30,3.0,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
127,8946.0,8946.0,1.0,0.0,67.0,578.0,94.0,1.74,31.05,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
128,8951.0,8951.0,1.0,0.0,67.0,52.0,65.0,1.74,21.47,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
129,8957.0,8957.0,1.0,0.0,74.0,269.0,83.0,1.65,30.49,4.0,...,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
130,8963.0,8963.0,0.0,0.0,79.0,232.0,84.0,1.78,26.51,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:


# Definir llindar per considerar una columna int64 com categòrica
umbral_categorico = 10

# Omplir  buids amb mitjana
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].fillna(df[col].mean())

# Omplir buids amb moda
for col in df.select_dtypes(include=['int64']).columns:
    if df[col].nunique() <= umbral_categorico:
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].mean())



In [ ]:
# Guardar el DataFrame modificado como un archivo CSV
#df.to_csv("df_mitjana_moda.csv", index=False)


IMPUTACIO KNN-IMPUTER a lo bruto

In [ ]:
df = pd.read_csv('huse.csv')

In [ ]:
# df preprocessing
df = df.drop(columns=['CT_PRETACE_DATE', 'TACE_DATE', 'Exam_date1', 'Exam_date2', 'LR 2', 'LR  3', 'LR4', 'LR5', '7-11_CRITERIA_CAT'])
start_index = df[df['ID'] == 8965].index[0]
df = df.loc[:start_index - 1]

# Definir llindar de NaN màxims
threshold = 0.65
min_non_null_count = int(threshold * len(df))


initial_columns = set(df.columns)

# Filtrar columnes
df = df.dropna(thresh=min_non_null_count, axis=1)


removed_columns = initial_columns - set(df.columns)

# Mostrar
print(f"Número de columnas eliminadas: {len(removed_columns)}")
print("Nombres de las columnas eliminadas:", list(removed_columns))

Número de columnas eliminadas: 21
Nombres de las columnas eliminadas: ['BCLC', 'Bilirrubina_directa', 'ECOG', 'HIV_+', 'Calcio', 'AntiHBs', 'Encefalopathy', 'Bilirrubina_inderecta', 'otherserology_positivie_result', 'Magnesio', 'TP', 'MELD_SCORE', 'Proteina_total', 'HBsAc', 'TTPa', 'MELDNa_SCORE', 'Fosforo', 'HIV_Serology', 'DOSE _DLP', 'Cloruro', 'LDH']


In [ ]:


df = df.replace(',', '.', regex=True).astype(float)

numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Convertir numèriques amb pocs valors unics a categoriques
for col in numeric_columns:

    if df[col].nunique() < 5:
        df[col] = df[col].astype('category')

print(df.dtypes)






ID              float64
CLAVE           float64
RECIST         category
GENDER         category
AGE_AT_TACE     float64
                 ...   
IVb            category
V              category
VI             category
VII            category
VIII           category
Length: 88, dtype: object


In [ ]:
import pandas as pd

non_numeric_columns = df.select_dtypes(exclude=['float64', 'int64']).columns

# Comprovació
if len(non_numeric_columns) > 0:
    print("Columnas no numèriques tobades:")
    print(non_numeric_columns)
else:
    print("Cap.")


Columnas no numèriques tobades:
Index(['RECIST', 'GENDER', 'HBV', 'HCV', 'Alcohol _abuse\n', 'Obesity',
       'disorders', 'Aflatoxin', 'Drug-abuse/addict', 'Smoke ', 'no_active_ex',
       'Diabetes', 'Hypertension', 'Cancer History', 'cancer_active',
       'B-block_treatment', 'Statin_treatment', 'Antiretroviral_treatment',
       'Cronic_Kidney_Insufficiency', 'Kidney_substitute_treatment',
       'Fibrosis_grade', 'ALBI_grade', 'Child_Pugh', 'HBsAg', 'HCV_Serology',
       'IMAGE_MODALITY_FOR_LIRADS', 'IV_CONTRAST',
       'TECHNICALLY_ACCEPTABLE_FOR_LIRADS',
       'MORPHOLICAL_CHANGES_OF_HEPATIC_CIRRHOSIS', 'PORTAL_HYPERTENSION_SIGNS',
       'VENOUS_PATENCY', 'LI-RADS_VALORATION_MODALITY', 'WIDESPREAD_DISEASE',
       'TIV', 'Tumoral_Distribution', 'I', 'II', 'III', 'IVa', 'IVb', 'V',
       'VI', 'VII', 'VIII'],
      dtype='object')


In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd

# Separar
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['category']).columns

# Imputació per numèriques (KNNImputer)
knn_imputer = KNNImputer(n_neighbors=5)
df_numeric_imputed = pd.DataFrame(knn_imputer.fit_transform(df[numeric_cols]), columns=numeric_cols)

# Imputaciócategòriques (moda)
df_categorical_imputed = df[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]))

# Concatenar dades
df_imputed = pd.concat([df_numeric_imputed, df_categorical_imputed], axis=1)

# Verificar que no hi hagi NaN
print(df_imputed.isna().sum())


ID                 0
CLAVE              0
AGE_AT_TACE        0
DAYS_CT_PRETACE    0
WEIGHT             0
                  ..
IVb                0
V                  0
VI                 0
VII                0
VIII               0
Length: 88, dtype: int64


In [ ]:

df_imputed.to_csv("df_knnimputer.csv", index=False)

In [ ]:
total_nan = df_imputed.isna().sum().sum()

print(f"Total de NaN en el DataFrame: {total_nan}")

Total de NaN en el DataFrame: 0


GOWER

In [ ]:
df = pd.read_csv('huse.csv')

In [ ]:
# Identifica columnes amb menys de 10 valors únics
columnes_categoriques = [col for col in df.columns if df[col].nunique() < 10]

# Converteix aquestes columnes a categòriques
for col in columnes_categoriques:
    df[col] = df[col].astype('category')

# Identifica la resta de columnes (no categòriques)
columnes_numeriques = [col for col in df.columns if col not in columnes_categoriques]

# Converteix columnes numèriques de format amb comes a numèrics
for col in columnes_numeriques:
    # Substitueix comes per punts (només funciona per columnes tipus object o text)
    df[col] = df[col].astype(str).str.replace(',', '.')
    # Converteix a numèric
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Valors no vàlids es converteixen a NaN

for columna in df.columns:
    print(f"Columna: {columna}, Tipus: {df[columna].dtype}")

Columna: ID, Tipus: int64
Columna: CLAVE, Tipus: int64
Columna: RECIST, Tipus: category
Columna: CT_PRETACE_DATE, Tipus: float64
Columna: GENDER, Tipus: category
Columna: TACE_DATE, Tipus: float64
Columna: AGE_AT_TACE, Tipus: int64
Columna: DAYS_CT_PRETACE, Tipus: int64
Columna: WEIGHT, Tipus: float64
Columna: HEIGHT, Tipus: float64
Columna: BMI, Tipus: float64
Columna: BMI_CATEGORY, Tipus: category
Columna: HBV, Tipus: category
Columna: HCV, Tipus: category
Columna: Alcohol _abuse
, Tipus: category
Columna: Obesity, Tipus: category
Columna: disorders, Tipus: category
Columna: Aflatoxin, Tipus: category
Columna: Drug-abuse/addict, Tipus: category
Columna: Smoke , Tipus: category
Columna: no_active_ex, Tipus: category
Columna: Diabetes, Tipus: category
Columna: Hypertension, Tipus: category
Columna: HIV_+, Tipus: category
Columna: Cancer History, Tipus: category
Columna: cancer_active, Tipus: category
Columna: B-block_treatment, Tipus: category
Columna: Statin_treatment, Tipus: category

In [ ]:
# df preprocessing
df = df.drop(columns=['CT_PRETACE_DATE', 'TACE_DATE', 'Exam_date1', 'Exam_date2', 'LR 2', 'LR  3', 'LR4', 'LR5', '7-11_CRITERIA_CAT', 'HBV', 'HBsAg', 'HCV_Serology'])
start_index = df[df['ID'] == 8965].index[0]
df = df.loc[:start_index - 1]
#Filtrar les variables amb molts de NaN
threshold = 0.65
min_non_null_count = int(threshold * len(df))

initial_columns = set(df.columns)

df = df.dropna(thresh=min_non_null_count, axis=1)

removed_columns = initial_columns - set(df.columns)

print(f"Número de columnas eliminadas: {len(removed_columns)}")
print("Nombres de las columnas eliminadas:", list(removed_columns))

Número de columnas eliminadas: 21
Nombres de las columnas eliminadas: ['BCLC', 'Bilirrubina_directa', 'ECOG', 'HIV_+', 'Calcio', 'AntiHBs', 'Encefalopathy', 'Bilirrubina_inderecta', 'otherserology_positivie_result', 'Magnesio', 'TP', 'MELD_SCORE', 'Proteina_total', 'HBsAc', 'TTPa', 'MELDNa_SCORE', 'Fosforo', 'HIV_Serology', 'DOSE _DLP', 'Cloruro', 'LDH']


In [ ]:
df_ple = df.copy()
for col in df_ple.select_dtypes(include=['float64']).columns:
    df_ple[col] = df_ple[col].fillna(df[col].mean())

for col in df_ple.select_dtypes(include= ['category']).columns:
    df_ple[col] = df_ple[col].fillna(df[col].mode()[0])

In [ ]:
# Converteix les columnes numèriques si cal
for col in df_ple.select_dtypes(include=['float64', 'int64']).columns:
    df_ple[col] = df_ple[col].astype('float')

In [ ]:
# Converteix les columnes categòriques a object
for col in df_ple.select_dtypes(include=['category']).columns:
    df_ple[col] = df_ple[col].astype('object')

In [ ]:
import gower

# Calcula la matriu de distàncies
dist_matrix = gower.gower_matrix(df_ple)

In [ ]:
import numpy as np
import pandas as pd

def impute_nan_with_neighbors(df, dist_matrix, k=5):
    # Copia per treballar en un DataFrame nou
    df_filled = df.copy()

    for idx, row in df.iterrows():
        for col in df.columns:
            if pd.isnull(row[col]):  # Només imputar si és NaN
                # Troba els índexs dels veïns més propers (excloent-se a si mateix)
                neighbor_indices = [i for i in np.argsort(dist_matrix[idx]) if i != idx][:k]

                # Selecciona els veïns
                neighbors = df.loc[neighbor_indices]

                # Imputació segons el tipus de dada
                if df[col].dtype in ['float64', 'int64']:
                    # Imputació numèrica: mitjana
                    value = neighbors[col].mean()
                    if not np.isnan(value):  # Si la mitjana és vàlida
                        if df[col].dtype == 'int64':
                            df_filled.loc[idx, col] = int(value)
                        else:
                            df_filled.loc[idx, col] = value
                    else:
                    # Si no hi ha veïns vàlids, utilitza la mitjana global de la columna
                        global_mean = df[col].mean()
                        df_filled.loc[idx, col] = global_mean
                else:
                    # Imputació categòrica: moda
                    mode_value = neighbors[col].mode()
                    if not mode_value.empty:  # Si la moda és vàlida
                        df_filled.loc[idx, col] = mode_value[0]
                    else:
                        df_filled.loc[idx, col] = df[col].mode()[0]

    return df_filled

# Aplica la imputació
df_filled = impute_nan_with_neighbors(df, dist_matrix)

# Comprova si hi ha NaN restants
if df_filled.isna().sum().sum() > 0:
    print("Encara queden NaN al DataFrame:")
    print(df_filled.isna().sum())
else:
    print("Tots els NaN s'han imputat correctament!")

Tots els NaN s'han imputat correctament!


In [ ]:
df_filled.head()

,ID,CLAVE,RECIST,GENDER,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
0,222,222,0,1,82,47,64.0,1.47,29.62,3.0,...,0,0,0,0,0,0,0,0,0,1
1,346,346,0,1,71,26,55.0,1.52,23.81,2.0,...,0,0,0,0,0,0,0,0,0,1
2,613,613,1,0,48,75,65.0,1.69,22.76,2.0,...,2,0,0,0,1,0,0,1,1,1
3,697,697,1,0,77,110,68.0,1.59,26.90,3.0,...,0,0,0,0,0,0,0,0,1,0
4,745,745,0,0,54,28,65.0,1.70,22.49,2.0,...,2,0,1,1,1,1,1,0,1,1


In [ ]:
df_filled.to_csv("df_gower.csv", index=False)

In [ ]:
total_nan = df_filled.isna().sum().sum()

print(f"Total de NaN en el DataFrame: {total_nan}")

Total de NaN en el DataFrame: 0
